In [42]:
import scipy.io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score
import torch
import numpy as np

In [43]:
# 分割数据集（无用）
def data_split(data, test_size):
    trn_data = {}
    val_data = {}

    for label, data in data.items():
        lens = len(data[label])
        print(lens)
        trn_data[label] = data[: lens * (1 - test_size)]
        val_data[label] = data[lens * (1 - test_size) :]
    
    return trn_data, val_data

In [44]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [45]:
# 读取mat
trn_mat = scipy.io.loadmat("data/train_data.mat")
trn_data = trn_mat["train"]
# print(len(trn_data))
x_len = len(trn_data)

# 标注数据label
trn_label = [i + 1 for i in range(x_len)]
# print(trn_label)

# 将label与data组合为字典
trn_dict = {}
for i in range(x_len):
    trn_dict[i+1] = trn_data[i]
# print(len(trn_dict))
print(type(trn_dict[1]))

# 读取测试集
val_mat = scipy.io.loadmat('data/test_data.mat')
val_data = val_mat['test']
val_data = val_data
print(type(val_data[1]))
print(len(val_data))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
1000


In [46]:
# 定义SVM
class LinearSVM:
    # 定义学习率、正则化强度、迭代次数
    def __init__(self, lr=0.01, reg=1e-5, num_iter=1000):
        self.lr = lr
        self.reg = reg
        self.num_iter = num_iter
        
    # 计算损失和梯度
    def compute_loss_and_grad(self, x, y):
        num_trn = x.shape[0]
        num_cls = np.max(y) # 样本类别的数量，在预处理数据时已+1，此处无需+1
        scores = x.dot(self.W)  # 获取全分数矩阵
        correct_cls_scores = scores[np.arange(num_trn), y] # 提取正确类别的分数 
        
        margins = np.maximum(0, scores - correct_cls_scores[:, np.newaxis] + 1) # 计算损失边缘
        margins[np.arange(num_trn), y] = 0  # 令正确分类损失为0
        loss = np.sum(margins) / num_trn    # 计算损失
        loss += 0.5 * self.reg * np.sum(self.W * self.W)    # 对损失进行正则化处理，防止过拟合
        
        margins[margins>0] = 1  # 将边缘损失矩阵大于零的部分设为1，方便计算梯度
        margins[np.arange(num_trn), y] = -np.sum(margins, axis=1)   
        dW = x.T.dot(margins) / num_trn # 计算权重矩阵的梯度取平均值，用于更新模型参数
        dW += self.reg * self.W # 正则化梯度，用于惩罚过大的权重
        
        return loss, dW

    # 训练
    def fit(self, x, y):
        num_trn, dim = x.shape
        num_cls = np.max(y) + 1
        self.W = 0.001 * np.random.randn(dim, num_cls)  # 生成一个初始权重矩阵

        for i in range(self.num_iter):
            loss, grad = self.compute_loss_and_grad(x, y)
            self.W -= self.lr * grad    # 梯度下降法更新权重
            if i % 100 == 0:
                print(f"Iteration {i}/{self.num_iter}, Loss: {loss}\n {grad[0].shape}")
                
    # 预测
    def predict(self, x):
        scores = x.dot(self.W)  # 利用权重进行预测
        if scores.size == 0:
            return []  # 如果输入序列为空，则返回空列表
        return np.argmax(scores, axis=1)    # 输出得分最高的类别

In [47]:
# 收集并转换数据
x_trn = []
y_trn = []
for y, x in trn_dict.items():
    x_trn.append(x)
    y_trn.append([y] * len(x))

x_trn = np.array(x_trn)
y_trn = np.array(y_trn)
print(x_trn.shape)

# 重塑数据
x_trn = x_trn.reshape(200 * 15, 28 * 28).astype(np.float64)
y_trn = np.repeat(np.arange(200), 15)
x_val = val_data.reshape(val_data.shape[0], -1).astype(np.float64)

# 计算并减去均值图像
mean_img = np.mean(x_trn, axis=0).astype(np.float64)
x_trn -= mean_img
x_val -= mean_img

# 训练 SVM
svm = LinearSVM()
svm.fit(x_trn, y_trn)

(200, 15, 28, 28)


ValueError: cannot reshape array of size 2352000 into shape (2200,784)

In [ ]:
# 预测
predictions = svm.predict(x_val)

print(len(x_val))
print(predictions)

1000
[120 172 194   0 135  48 194 134  13  28  91 153 170 147 121  94 116  12
 183  68 148 180  39  13 177  91 102   1 199  25  94 155  36 118   5  74
 176  18 176 187  65   9  43   3 111  36  79  84 189  75  75 125 163   8
 123  70   9 145  56  31  57  24 137 167  81 119 118  47  77 170  36 111
 162 116  14 119 182  55 134 137  33 158 147  64   8 163  79 177  90 184
  44 119 104 162 101  41 138  71 127   2 155  56 155  71  27 171  52  85
  32  16  31   9  30  68 130 194   0  35  40 190 193  70 131  27  90  47
  77 123 100 112  82 194  27 123 130 162 104  52 165 188  92 162 177 132
 184  32 121 184 121 154 171 166  50 112 157 104 110  53 127  79 171  21
 154 152  77 127  84  18  45 142  87 138   9  44  32  31 156 179 131 173
 171  45 100 144  72  74  24  43 118 128 154  46 186  90  94  92  40  14
 124  92  26  81 156 162  73 149 189  66 101 164 171 115 128  41  94  93
 121 193   3  84 158 118  91 166  19 187 162  92 156  78  92 168 110  54
 169 166   5 177 147  44  17 109 138 173 159  

In [ ]:
# 将预测结果写入csv
import pandas as pd

submission = pd.read_csv('data/submission.csv', encoding='utf8')
submission['预测结果'] = predictions
submission.to_csv('data/submission.csv', index=False)